
**Correr un Servidor de Minecraft en Google Colab!**

---

# Iniciar el servidor ya creado

El codigo siguiente iniciara el servidor si ya esta creado.

In [ ]:
import os
import re
import json

# Actualizar la lista de paquetes
!sudo apt update &>/dev/null && echo "apt actualizado correctamente" || echo "error al actualizar apt"

# Remover java 11
!sudo apt remove -y openjdk-11-jdk-headless openjdk-11-jre-headless &>/dev/null && echo "java 11 eliminado correctamente" || echo "error al eliminar java 11"
!sudo apt autoremove -y &>/dev/null && echo "paquetes limpiados correctamente" || echo "error al limpiar paquetes"

# Instalar java 21
!sudo apt install -y wget apt-transport-https &>/dev/null && echo "instalado wget y apt-transport-https" || echo "error al instalar wget y apt-transport-https"
!mkdir -p /etc/apt/keyrings &>/dev/null && echo "creando directorio de claves" || echo "error al crear directorio de claves"
!wget -O - https://packages.adoptium.net/artifactory/api/gpg/key/public | tee /etc/apt/keyrings/adoptium.asc &>/dev/null && echo "obteniendo claves" || echo "error al obtener claves"
!echo "deb [signed-by=/etc/apt/keyrings/adoptium.asc] https://packages.adoptium.net/artifactory/deb $(awk -F= '/^VERSION_CODENAME/{print$2}' /etc/os-release) main" | tee /etc/apt/sources.list.d/adoptium.list &>/dev/null && echo "claves agregadas" || echo "error al agregar claves"
!sudo apt update &>/dev/null && echo "apt actualizado correctamente" || echo "error al actualizar apt"
!sudo apt install temurin-21-jdk &>/dev/null && echo "java 21 se instaló correctamente." || echo "error al instalar java 21."

# Limpiar la lista de paquetes
!sudo apt clean &>/dev/null && echo "paquetes limpiados correctamente" || echo "error al limpiar paquetes"

# Chequear la version de java
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'

if java_ver[0] == "21" :
  print("java 21 esta funcionando correctamente, podes seguir.")

  # Montar Google Drive
  from google.colab import drive
  drive.mount('/content/drive')
  
  # Cambiar directorio al de Minecraft en Google Drive
  %cd "/content/drive/My Drive/servidor-minecraft"

  !ls #listar contenido del directorio (para verificar que cambió el directorio de trabajo)

  # Importar archivo de configuracion.
  if os.path.isfile("colabconfig.json"):
    colabconfig = json.load(open("colabconfig.json"))
  else:
    colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
    json.dump(colabconfig, open("colabconfig.json",'w'))

  # Nombres de los archivos jar de servidor.
  jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'server.jar'}
  jar_name = jar_list[colabconfig["server_type"]]

  # Argumentos java.
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
  memory_allocation = "-Xms6G -Xmx6G"

  # Elegir el servicio de tunel para usar
  # Opciones disponibles: ngrok, playit
  tunnel_service = "ngrok"
  print("Procediendo a usar", tunnel_service)

  if tunnel_service == "ngrok":
    !pip -q install pyngrok
    from pyngrok import conf, ngrok

    # Preguntar por el token de ngrok
    print("Obtené tu token de https://dashboard.ngrok.com/auth")
    import getpass
    authtoken = getpass.getpass()  # Coloca el token de ngrok siempre que inicies o remplazalo
    !ngrok authtoken $authtoken # iniciar sesion en ngrok

    # Establecer la region de ngrok
    conf.get_default().region = 'sa'  # Cambia esto a la region que quieras, en este caso es Sudamerica

    # Conectar a ngrok
    url = ngrok.connect(25565, 'tcp')
    print('La dirección de tu servidor es ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))
    print('Iniciando servidor...')
    !java $memory_allocation $server_flags -jar $jar_name nogui

  elif tunnel_service == "playit":
    !curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
    !sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
    !sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg instalado" || echo "Falló al instalar playit"
    print('Iniciando servidor...')
    !playit & java $memory_allocation $server_flags -jar $jar_name nogui
else:
  print("java 21 no parece estar instalado o no esta funcionando")

# Crear un servidor

El codigo siguiente descargara un servidor y aceptara el EULA. Luego ya podes iniciarlo

In [ ]:
# Remplaza "1.20.2" con la version que quieras usar.
# Versiones disponibles:
# - 1.20.2
# - 1.20.1
# - 1.20
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - 1.17
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8
# Las versiones nuevas puede que funcionen, pero no esta garantizado.
version = '1.20.2'
forge_version = "43.2.8"  # Si tu server usa forge

#Elegi el tipo de servidor. Versiones disponibles: fabric, paper, forge
server_type = 'paper'

from google.colab import drive
import requests
import json

drive.mount('/content/drive')
!mkdir "/content/drive/My Drive/servidor-minecraft"
%cd "/content/drive/My Drive/servidor-minecraft"

if server_type == "paper":
  a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)
  b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))
  print("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"])
  url = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]
  jar_name = "server.jar"
  print('Descargando a Google Drive...')
elif server_type == "fabric":
  url = "https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.9.1/fabric-installer-0.9.1.jar"
  jar_name = "fabric-installer.jar"
elif server_type == "forge":
  url = f"https://maven.minecraftforge.net/net/minecraftforge/forge/{version}-{forge_version}/forge-{version}-{forge_version}-installer.jar"
  jar_name = "server.jar"

r = requests.get(url)
if r.status_code == 200:
  with open('/content/drive/My Drive/servidor-minecraft/' + jar_name, 'wb') as f:
    f.write(r.content)
else:
  print('Error '+ str(r.status_code) + '! Parece que pusiste una version no soportada. Intentá ejecutar el codigo otra vez.')

# Ejecutando una dirección de instalacion especifica.
if server_type == "fabric":
  !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft
elif server_type == "forge":
  !java -jar server.jar --installServer

colabconfig = {"server_type": server_type}
json.dump(colabconfig, open("colabconfig.json",'w'))

# Aceptar el EULA
%cd "/content/drive/My Drive/servidor-minecraft"
!echo "eula=true" >> eula.txt

print('Listo!')